In [20]:
import pandas as pd 
import numpy as ap
from datetime import timedelta
import numpy as np
from pycaret.regression import *

In [21]:
df_future = pd.read_csv('file_predict\predict_jsps014.h.csv')

In [22]:
last_date = df_future["timestamp"].max()
last_date = pd.to_datetime(last_date)

In [44]:
future_date = pd.date_range(start=last_date + timedelta(days=1), periods=8, freq="D")
future_date

DatetimeIndex(['2025-02-19 11:12:43', '2025-02-20 11:12:43',
               '2025-02-21 11:12:43', '2025-02-22 11:12:43',
               '2025-02-23 11:12:43', '2025-02-24 11:12:43',
               '2025-02-25 11:12:43', '2025-02-26 11:12:43'],
              dtype='datetime64[ns]', freq='D')

In [45]:
future_dates = pd.DataFrame({"timestamp": future_date})
future_dates.head()

,timestamp
0,2025-02-19 11:12:43
1,2025-02-20 11:12:43
2,2025-02-21 11:12:43
3,2025-02-22 11:12:43
4,2025-02-23 11:12:43


In [46]:
Q1_H = df_future["humidity"].quantile(0.25)
Q3_H = df_future["humidity"].quantile(0.75)
IQR_H = Q3_H - Q1_H

lower_bound_H = max(0, Q1_H - 1.5 * IQR_H) 
upper_bound_H = Q3_H + 1.5 * IQR_H 

Q1_TT = df_future["temperature"].quantile(0.25)
Q3_TT = df_future["temperature"].quantile(0.75)
IQR_TT = Q3_TT - Q1_TT

lower_bound_TT = max(0, Q1_TT - 1.5 * IQR_TT) 
upper_bound_TT = Q3_TT + 1.5 * IQR_TT 


In [47]:
future_dates["month"] = future_dates["timestamp"].dt.month
future_dates["dayofweek"] = future_dates["timestamp"].dt.dayofweek
future_dates["weekofyear"] = future_dates["timestamp"].dt.dayofyear
future_dates["day"] = future_dates["timestamp"].dt.day
future_dates["hour"] = future_dates["timestamp"].dt.hour


pm2_5_mean = df_future["temperature"].mean()
pm2_5_std = df_future["temperature"].std()

hum_mean = df_future["humidity"].mean()
hum_std = df_future["humidity"].std()

future_dates["humidity"] = np.random.normal(hum_mean, hum_std, len(future_dates))
future_dates["temperature"] = np.random.normal(pm2_5_mean, pm2_5_std, len(future_dates))

future_dates["humidity"] = future_dates["humidity"].clip(lower=lower_bound_H, upper=upper_bound_H)
future_dates["temperature"] = future_dates["temperature"].clip(lower=lower_bound_TT, upper=upper_bound_TT)
future_dates

,timestamp,month,dayofweek,weekofyear,day,hour,humidity,temperature
0,2025-02-19 11:12:43,2,2,50,19,11,61.655373,29.375987
1,2025-02-20 11:12:43,2,3,51,20,11,79.875741,29.752132
2,2025-02-21 11:12:43,2,4,52,21,11,54.616266,34.071473
3,2025-02-22 11:12:43,2,5,53,22,11,62.695129,28.263133
4,2025-02-23 11:12:43,2,6,54,23,11,78.419076,25.719408
5,2025-02-24 11:12:43,2,0,55,24,11,73.990665,28.935934
6,2025-02-25 11:12:43,2,1,56,25,11,57.634717,26.636770
7,2025-02-26 11:12:43,2,2,57,26,11,56.732750,29.247526


In [27]:
df = pd.concat([df_future,future_dates], ignore_index=True)

In [42]:
df['pm_2_5_lag_12day'] = df['pm_2_5'].shift(336)
df['PM2.5_MA3_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).mean()
df['PM2.5_MAX24_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).max()
df['PM2.5_MIN24_prev_12day'] = df['pm_2_5'].shift(336).rolling(window=336).min()
df.tail(12)

,Unnamed: 0.1,Unnamed: 0,timestamp,humidity,pm_2_5,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
23929,23929.0,NaN,2025-02-14 11:12:43,61.579528,NaN,31.230701,2,4,45,14,11,10.950413,20.488655,47.341667,1.133333
23930,23930.0,NaN,2025-02-15 11:12:43,90.161433,NaN,29.851736,2,5,46,15,11,11.445378,20.419579,47.341667,1.133333
23931,23931.0,NaN,2025-02-16 11:12:43,76.656375,NaN,29.331557,2,6,47,16,11,10.735043,20.352135,47.341667,1.133333
23932,23932.0,NaN,2025-02-17 11:12:43,65.767412,NaN,28.004407,2,0,48,17,11,12.066116,20.288331,47.341667,1.133333
23933,23933.0,NaN,2025-02-18 11:12:43,59.402024,NaN,24.984442,2,1,49,18,11,10.974790,20.217553,47.341667,1.133333
23934,NaN,NaN,2025-02-19 11:12:43,67.135726,NaN,30.251017,2,2,50,19,11,10.725000,20.144740,47.341667,1.133333
23935,NaN,NaN,2025-02-20 11:12:43,82.102136,NaN,31.136358,2,3,51,20,11,10.446281,20.068092,47.341667,1.133333
23936,NaN,NaN,2025-02-21 11:12:43,97.600312,NaN,30.435310,2,4,52,21,11,9.983193,19.990430,47.341667,1.133333
23937,NaN,NaN,2025-02-22 11:12:43,74.095603,NaN,26.445683,2,5,53,22,11,10.327586,19.905292,47.341667,1.133333
23938,NaN,NaN,2025-02-23 11:12:43,68.247312,NaN,31.576620,2,6,54,23,11,9.491667,19.799363,47.341667,1.133333


In [29]:
df.drop(columns=(["Unnamed: 0"]))

,Unnamed: 0.1,timestamp,humidity,pm_2_5,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
0,0.0,2022-05-01 17:00:00,74.810899,7.205128,28.220940,5,6,121,1,17,NaN,NaN,NaN,NaN
1,1.0,2022-05-01 18:00:00,73.963971,9.208333,28.551333,5,6,121,1,18,NaN,NaN,NaN,NaN
2,2.0,2022-05-01 19:00:00,74.998299,12.883333,28.158917,5,6,121,1,19,NaN,NaN,NaN,NaN
3,3.0,2022-05-01 20:00:00,76.198977,15.483333,28.019250,5,6,121,1,20,NaN,NaN,NaN,NaN
4,4.0,2022-05-01 21:00:00,78.530695,12.400000,27.682333,5,6,121,1,21,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23936,NaN,2025-02-21 11:12:43,97.600312,NaN,30.435310,2,4,52,21,11,9.983193,NaN,NaN,NaN
23937,NaN,2025-02-22 11:12:43,74.095603,NaN,26.445683,2,5,53,22,11,10.327586,NaN,NaN,NaN
23938,NaN,2025-02-23 11:12:43,68.247312,NaN,31.576620,2,6,54,23,11,9.491667,NaN,NaN,NaN
23939,NaN,2025-02-24 11:12:43,70.338819,NaN,34.909105,2,0,55,24,11,10.733333,NaN,NaN,NaN


In [30]:
df_predict=df.tail(7)
df_predict.drop(columns=(["Unnamed: 0","pm_2_5"]),inplace=True)
df_predict

,Unnamed: 0.1,timestamp,humidity,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day
23934,NaN,2025-02-19 11:12:43,67.135726,30.251017,2,2,50,19,11,10.725000,NaN,NaN,NaN
23935,NaN,2025-02-20 11:12:43,82.102136,31.136358,2,3,51,20,11,10.446281,NaN,NaN,NaN
23936,NaN,2025-02-21 11:12:43,97.600312,30.435310,2,4,52,21,11,9.983193,NaN,NaN,NaN
23937,NaN,2025-02-22 11:12:43,74.095603,26.445683,2,5,53,22,11,10.327586,NaN,NaN,NaN
23938,NaN,2025-02-23 11:12:43,68.247312,31.576620,2,6,54,23,11,9.491667,NaN,NaN,NaN
23939,NaN,2025-02-24 11:12:43,70.338819,34.909105,2,0,55,24,11,10.733333,NaN,NaN,NaN
23940,NaN,2025-02-25 11:12:43,76.758888,33.553169,2,1,56,25,11,12.258333,NaN,NaN,NaN


In [31]:
loaded_Extra_Trees_Regressor = load_model('pm25_Extra_Trees_Regressor')

Transformation Pipeline and Model Successfully Loaded


In [32]:
predict = predict_model(loaded_Extra_Trees_Regressor,data=df_predict)
predict

,Unnamed: 0.1,timestamp,humidity,temperature,month,dayofweek,weekofyear,day,hour,pm_2_5_lag_12day,PM2.5_MA3_prev_12day,PM2.5_MAX24_prev_12day,PM2.5_MIN24_prev_12day,prediction_label
23934,NaN,2025-02-19 11:12:43,67.135727,30.251017,2,2,50,19,11,10.725000,NaN,NaN,NaN,12.469181
23935,NaN,2025-02-20 11:12:43,82.102135,31.136358,2,3,51,20,11,10.446281,NaN,NaN,NaN,17.085204
23936,NaN,2025-02-21 11:12:43,97.600311,30.435310,2,4,52,21,11,9.983193,NaN,NaN,NaN,17.561455
23937,NaN,2025-02-22 11:12:43,74.095604,26.445683,2,5,53,22,11,10.327586,NaN,NaN,NaN,13.845886
23938,NaN,2025-02-23 11:12:43,68.247314,31.576620,2,6,54,23,11,9.491667,NaN,NaN,NaN,13.085060
23939,NaN,2025-02-24 11:12:43,70.338821,34.909103,2,0,55,24,11,10.733334,NaN,NaN,NaN,14.587694
23940,NaN,2025-02-25 11:12:43,76.758888,33.553169,2,1,56,25,11,12.258333,NaN,NaN,NaN,16.606860
